In [ ]:
install.packages("RPostgreSQL", lib="/opt/app-root/src/Rpackages")
install.packages("RPostgres", lib="/opt/app-root/src/Rpackages")


In [ ]:
db <- "postgres"
db_host <- Sys.getenv("PGHOST")
db_port <- Sys.getenv("PGPORT")
db_user <- Sys.getenv("PGUSER")
db_pass <- Sys.getenv("PGPASSWORD")
print("DONE")

In [ ]:
library(DBI)
conn <- dbConnect(
  RPostgres::Postgres(),
  dbname = "bety",
  host = db_host,
  port = db_port,
  user = db_user,
  password = db_pass
)
print("DONE")

In [ ]:
dbGetQuery(conn, "SELECT sites.id, sites.sitename, sites.city, sites.country, ST_X(ST_CENTROID(sites.geometry)) AS lon, ST_Y(ST_CENTROID(sites.geometry)) AS lat FROM sites limit 5")

In [ ]:
library(shiny)
library(glue)
library(leaflet)

ui <- fluidPage(
  tags$h1("Ecological Forecasting Sites"),
  leafletOutput(outputId = "map")
)


In [ ]:
server <- function(input, output) {
  data <- reactive({
    conn <- dbConnect(
      RPostgres::Postgres(),
      dbname = "bety",
      host = db_host,
      port = db_port,
      user = db_user,
      password = db_pass
    )
    sites <- dbGetQuery(conn, "SELECT sites.id, sites.sitename, sites.city, sites.country, ST_X(ST_CENTROID(sites.geometry)) AS lon, ST_Y(ST_CENTROID(sites.geometry)) AS lat FROM sites INNER JOIN sitegroups_sites ON sitegroups_sites.site_id=sites.id where sitegroups_sites.sitegroup_id=1")
    dbDisconnect(conn)
    data.frame(sites)
  })

  output$map <- renderLeaflet({
    leaflet(data = data()) %>%
      addTiles() %>%
      addMarkers(~lon, ~lat, label = ~sitename) %>%
      addProviderTiles(providers$Esri.WorldStreetMap)
  })
}
print("DONE")

In [ ]:
app <- shinyApp(ui = ui, server)
shiny::runApp(host = '0.0.0.0', port=8080, app=app)



## Visit your RShiny app in development

Click on the link above to access the RShiny app in development.
